### 请仔细阅读相关文档并补充完整下面的代码。在需要补充的部分已经标注#TODO并附上相应的内容提示。

In [4]:
pip install tensorboard

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 9.9 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 5.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
# 导入必要的库
import torch  # PyTorch 深度学习框架
import torch.nn as nn  # 神经网络相关模块
import numpy as np  # 数值计算库
from torch.utils.data import DataLoader  # 处理数据加载
import torchvision
from torchvision import datasets, transforms  # 处理图像数据集和数据变换
from torchvision.utils import save_image  # 保存生成的图像
import os  # 处理文件和目录操作
from torch.utils.tensorboard import SummaryWriter  # TensorBoard

In [7]:
import torch
torch.cuda.is_available()


False

#### 根据文档和提示，补充完整生成器Generator和判别器Discriminator代码：

In [8]:
# =============================== 生成器（Generator） ===============================
class Generator(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            #TODO
            nn.Linear(input_dim, hidden_dim),           # 使用线性层将随机噪声映射到第一个隐藏层
            nn.ReLU(),      # 使用 ReLU 作为激活函数，帮助模型学习非线性特征
            #TODO
            nn.Linear(hidden_dim, hidden_dim),           # 使用线性层将第一个隐藏层映射到第二个隐藏层
            nn.ReLU(),      # 再次使用 ReLU 激活函数
            #TODO
            nn.Linear(hidden_dim, output_dim),           # 使用线性层将第二个隐藏层映射到输出层，输出为图像的像素大小
            nn.Tanh()       # 使用 Tanh 将输出归一化到 [-1, 1]，适用于图像生成
        )

    def forward(self, x):
        #TODO               # 前向传播：将输入 x 通过模型进行计算，得到生成的图像
        return self.model(x)

# =============================== 判别器（Discriminator） ===============================
class Discriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            #TODO   # 输入层到第一个隐藏层，使用线性层
            nn.Linear(input_dim, hidden_dim),
            #TODO   # 使用 LeakyReLU 激活函数，避免梯度消失问题，negative_slope参数设置为0.1
            nn.LeakyReLU(negative_slope=0.1),
            #TODO   # 第一个隐藏层到第二个隐藏层，使用线性层
            nn.Linear(hidden_dim, hidden_dim),
            #TODO   # 再次使用 LeakyReLU 激活函数，negative_slope参数设置为0.1
            nn.LeakyReLU(negative_slope=0.1),
            #TODO   # 第二个隐藏层到输出层，使用线性层
            nn.Linear(hidden_dim, input_dim),
            #TODO   # 使用 Sigmoid 激活函数，将输出范围限制在 [0, 1]
            nn.Sigmoid()
        )

    def forward(self, x):
        #TODO       # 前向传播：将输入 x 通过模型进行计算，得到判别结果
        return self.model(x)

#### 补充完整主函数，在主函数中完成以下过程：
1. 数据加载：
加载并预处理数据集。对于GAN的训练，通常需要将数据集转换为张量格式，并进行适当的归一化。
2. 模型初始化：
创建生成器和判别器模型实例，并将它们移动到合适的设备（如GPU）上。
3. 优化器和损失函数定义：
为生成器和判别器分别定义优化器（如Adam），并设置适当的学习率和其他超参数。
定义损失函数（如二元交叉熵损失）用于评估模型性能。
4. 训练循环：
迭代多个epoch进行训练。在每个epoch中，遍历数据集并进行以下操作：
   * 训练判别器：使用真实数据和生成的假数据更新判别器的参数。
   * 训练生成器：通过生成假数据并试图欺骗判别器来更新生成器的参数。
   * 记录损失值到TensorBoard，以监控训练过程。
5. 结果保存：
在每个epoch结束时，生成一些示例图像并保存到TensorBoard，以便观察生成器的进展。


In [ ]:
# =============================== 主函数 ===============================
def main():

    # 设备配置：使用 GPU（如果可用），否则使用 CPU
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # 设置模型和训练的超参数
    input_dim = 100  # 生成器输入的随机噪声向量维度
    hidden_dim = 256  # 隐藏层维度
    output_dim = 28 * 28  # MNIST 数据集图像尺寸（28x28）
    batch_size = 128  # 训练时的批量大小
    num_epoch = 30 # 训练的总轮数
    
    # 加载 MNIST 数据集
    train_dataset = datasets.MNIST(root="./data/", train=True, transform=transforms.ToTensor(), download=True)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    # 创建生成器G和判别器D，并移动到 GPU（如果可用）
    #TODO   # 生成器G
    G = Generator(input_dim=input_dim, hidden_dim= hidden_dim, output_dim= output_dim).to(device)
    #TOOD   # 判别器D
    D = Discriminator(input_dim= output_dim, hidden_dim= hidden_dim).to(device)

    # 定义针对生成器G的优化器optim_G和针对判别器D的优化器optim_D，要求使用Adam优化器，学习率设置为0.0002
    #TODO  # 生成器优化器optim_G
    optim_G = torch.optim.Adam(G.parameters(), lr=0.0002)
    #TODO  # 判别器优化器optim_D
    optim_D = torch.optim.Adam(D.parameters(), lr=0.0002)

    loss_func = nn.BCELoss()  # 使用二元交叉熵损失

    # 初始化 TensorBoard
    writer = SummaryWriter(log_dir='./logs/experiment_gan')

    # 开始训练
    for epoch in range(num_epoch):
        total_loss_D, total_loss_G = 0, 0
        for i, (real_images, _) in enumerate(train_loader):
            loss_D = train_discriminator(real_images, D, G, loss_func, optim_D, batch_size, input_dim, device)  # 训练判别器
            loss_G = train_generator(D, G, loss_func, optim_G, batch_size, input_dim, device)  # 训练生成器

            total_loss_D += loss_D
            total_loss_G += loss_G

            # 每 100 步打印一次损失
            if (i + 1) % 100 == 0 or (i + 1) == len(train_loader):
                print(f'Epoch {epoch:02d} | Step {i + 1:04d} / {len(train_loader)} | Loss_D {total_loss_D / (i + 1):.4f} | Loss_G {total_loss_G / (i + 1):.4f}')

            # 记录每个epoch的平均损失到 TensorBoard
        writer.add_scalar('GAN/Loss/Discriminator', total_loss_D / len(train_loader), epoch)
        writer.add_scalar('GAN/Loss/Generator', total_loss_G / len(train_loader), epoch)

        # 生成并保存示例图像
        with torch.no_grad():
            noise = torch.randn(64, input_dim, device=device)
            fake_images = G(noise).view(-1, 1, 28, 28)  # 调整形状为图像格式

            # 记录生成的图像到 TensorBoard
            img_grid = torchvision.utils.make_grid(fake_images, normalize=True)
            writer.add_image('Generated Images', img_grid, epoch)

#### 根据文档中描述的GAN的损失函数和二元交叉熵损失相关内容，补充完善Discriminator和Generator的训练过程：

In [44]:
# =============================== 训练判别器 ===============================
def train_discriminator(real_images, D, G, loss_func, optim_D, batch_size, input_dim, device):
    '''训练判别器'''
    real_images = real_images.view(-1, 28 * 28).to(device)  # 获取真实图像并展平
    real_output = D(real_images)  # 判别器预测真实图像
    #TODO   # 计算真实样本的损失real_loss
    real_loss = loss_func(real_output, real_images)
    noise = torch.randn(real_images.shape[0], input_dim, device=device)  # 生成随机噪声
    fake_images = G(noise).detach()  # 生成假图像（detach 避免梯度传递给 G）
    fake_output = D(fake_images)  # 判别器预测假图像
    
    #TODO   # 计算假样本的损失fake_loss
    fake_loss = loss_func(fake_output, real_images)

    loss_D = real_loss + fake_loss  # 判别器总损失
    optim_D.zero_grad()  # 清空梯度
    loss_D.backward()  # 反向传播
    optim_D.step()  # 更新判别器参数

    return loss_D.item()  # 返回标量损失

# =============================== 训练生成器 ===============================
def train_generator(D, G, loss_func, optim_G, batch_size, input_dim, device):
    '''训练生成器'''
    noise = torch.randn(batch_size, input_dim, device=device)  # 生成随机噪声
    fake_images = G(noise)  # 生成假图像
    fake_output = D(fake_images)  # 判别器对假图像的判断
    #TODO # 计算生成器损失（希望生成的图像判别为真）
    loss_G = loss_func(fake_output, torch.ones_like(fake_images))
    optim_G.zero_grad()  # 清空梯度
    loss_G.backward()  # 反向传播
    optim_G.step()  # 更新生成器参数

    return loss_G.item()  # 返回标量损失

In [36]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
modeltest = torch.nn.Linear(10,10).to(device)
x = torch.randn(5,10).to(device)
y = modeltest(x)
print(y.device)

cpu


#### 程序执行入口

In [46]:
if __name__ == '__main__':
    main()

Epoch 00 | Step 0100 / 469 | Loss_D 1.0594 | Loss_G 0.7544
Epoch 00 | Step 0200 / 469 | Loss_D 0.8989 | Loss_G 0.9729
Epoch 00 | Step 0300 / 469 | Loss_D 0.7953 | Loss_G 1.3594
Epoch 00 | Step 0400 / 469 | Loss_D 0.7283 | Loss_G 1.6448
Epoch 00 | Step 0469 / 469 | Loss_D 0.6942 | Loss_G 1.8257
Epoch 01 | Step 0100 / 469 | Loss_D 0.4844 | Loss_G 2.8986
Epoch 01 | Step 0200 / 469 | Loss_D 0.4751 | Loss_G 3.0111
Epoch 01 | Step 0300 / 469 | Loss_D 0.4684 | Loss_G 3.1221
Epoch 01 | Step 0400 / 469 | Loss_D 0.4627 | Loss_G 3.2538
Epoch 01 | Step 0469 / 469 | Loss_D 0.4597 | Loss_G 3.3253
Epoch 02 | Step 0100 / 469 | Loss_D 0.4350 | Loss_G 3.9378
Epoch 02 | Step 0200 / 469 | Loss_D 0.4323 | Loss_G 3.8840
Epoch 02 | Step 0300 / 469 | Loss_D 0.4305 | Loss_G 3.8748
Epoch 02 | Step 0400 / 469 | Loss_D 0.4283 | Loss_G 3.9348
Epoch 02 | Step 0469 / 469 | Loss_D 0.4268 | Loss_G 3.9590
Epoch 03 | Step 0100 / 469 | Loss_D 0.4135 | Loss_G 4.0503
Epoch 03 | Step 0200 / 469 | Loss_D 0.4113 | Loss_G 4.06